In [1]:
def GetResponseAttr(attr_name, resp_text, quote = '"'):
    attr_name = attr_name+'='+quote
    start = resp_text.find(attr_name)+len(attr_name)
    end = resp_text.find(quote, start)
    return resp_text[start: end]

In [2]:
def GetResponseDict(resp_text):
    #возвращает только первый словарь.
    #будут проблемы, если будет вложенный словарь
    start = resp_text.find('{')
    end = resp_text.find('}', start)
    return resp_text[start: end+1]

In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)
import requests
captcha_link ='https://captcha.kik.com/?id='
captcha_id = '3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA'

ses = requests.session()
res = ses.get(captcha_link+captcha_id)

next1 ='https://funcaptcha.com/fc/api/?onload=callbackCaptchaReady'
res1 = ses.get(next1)
#get next link
next2 = GetResponseAttr('t.src',res1.text)
res2 = ses.get(next2)
#get next link
GetCaptchaTokenDict = {'bda':'W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0seyJrZXkiOiJwIiwidmFsdWUiOjF9L'\
                       'Hsia2V5IjoiZiIsInZhbHVlIjoiNzY1MWVkMTE5ZmRmZDhlNWQzMTA0NGYzZjk3N2ZkMDIifSx7Im'\
                       'tleSI6ImNzIiwidmFsdWUiOjF9XQ=='\
                       ,'public_key':'92A09218-4AD3-CC2B-8145-87CD0B851124'\
                       ,'site':'https://captcha.kik.com'\
                       ,'data[kik_id]':captcha_id}
            
next3 =  GetResponseAttr('var S7e', res2.text)\
        +GetResponseAttr('var Z8e', res2.text)\
        +'&'+'&'.join(map(lambda x: '{}={}'.format(x[0],x[1]), GetCaptchaTokenDict.items()))
        
res3 = ses.get(next3)
#токен получен успешно.

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): captcha.kik.com
DEBUG:urllib3.connectionpool:https://captcha.kik.com:443 "GET /?id=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): funcaptcha.com
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/api/?onload=callbackCaptchaReady HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.funcaptcha.com
DEBUG:urllib3.connectionpool:https://cdn.funcaptcha.com:443 "GET /fc/js/ba1e486f2d32eee8fdf97ee89ec06b14/standard/funcaptcha_api.js HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/gt2/?callback=fcJSONPCallback&bda=W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0s

In [4]:
next4 = 'https://funcaptcha.com/fc/gc/'+'?token='\
        +eval(GetResponseDict(res3.text)).get('token').replace('\\/','/').replace('|','&')\
        +'&ps='+captcha_id

res4 = ses.get(next4)

next5 = 'https://funcaptcha.com/fc/apps/canvas/001/'
res5 = ses.get(next5, headers={'referer': next4})

next6 = 'https://funcaptcha.com/fc/a/'
res6 = ses.get(next6, headers={'referer': next4})
print(res6.text)

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/gc/?token=8135a33e202af59b2.88290908&r=eu-west-1&metabgclr=transparent&metatimeclr=%237a7d8e&meta=6&guitextcolor=%2326beff&metaiconclr=transparent&pk=92A09218-4AD3-CC2B-8145-87CD0B851124&injs=https://cdn.funcaptcha.com/fc/assets/graphics/kik/scripts/kik_help_8.js&at=40&ps=id&ht=1&cdn_url=https://cdn.funcaptcha.com/fc&surl=https://funcaptcha.com&ps=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/apps/canvas/001/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/a/ HTTP/1.1" 200 None


{"logged":false}


In [5]:
next7 = 'https://funcaptcha.com/fc/ekey/'
res7 = ses.post(next7, headers={'referer': next4})
print(res7.text)

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/ekey/ HTTP/1.1" 200 None


{"error":"DENIED ACCESS"}
